# Рекомендация тарифов

Входные данные - данные о поведении клиентов, которые уже перешли на эти тарифы "Смарт" и "Ультра" (из проекта курса «Статистический анализ данных»). 

Задача - построить модель для задачи классификации, которая выберет подходящий тариф (из тарифов "Смарт" и "Ультра") для клиентов со старыми тарифами. Достичь accuracy не меньше 0.75.

## Оглавление

* [1. Откройте и изучите файл](#1.-Откройте-и-изучите-файл)
    * [Выводы (шаг 1)](#Выводы-(шаг-1))
* [2. Разбейте данные на выборки](#2.-Разбейте-данные-на-выборки)
    * [Выводы (шаг 2)](#Выводы-(шаг-2))
* [3. Исследуйте модели](#3.-Исследуйте-модели)
    * [Логистическая регрессия](#Логистическая-регрессия)
    * [Дерево решений](#Дерево-решений)
    * [Случайный лес](#Случайный-лес)
    * [Выводы (шаг 3)](#Выводы-(шаг-3))
* [4. Проверьте модель на тестовой выборке](#4.-Проверьте-модель-на-тестовой-выборке)
    * [Выводы (шаг 4)](#Выводы-(шаг-4))
* [5. (бонус) Проверьте модели на адекватность](#5.-(бонус)-Проверьте-модели-на-адекватность)
    * [Выводы (шаг 5)](#Выводы-(шаг-5))

## 1. Откройте и изучите файл

In [342]:
import pandas as pd

In [343]:
data = pd.read_csv('users_behavior.csv')

In [344]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [345]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [346]:
# Cast to an integer type to save memory
data['calls'] = data['calls'].astype('int')
data['messages'] = data['messages'].astype('int')

data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


### Выводы (шаг 1)

Файл с данными состоит из 5 столбцов (1 целочисленный и 4 с плавающей точкой) и 3214 записей.

В столбце __is_ultra__ содержится целевой признак. 

Тип столбцов __calls__ и __messages__ был изменен на целочисленный для экономии памяти. Какая-либо другая предобработка отсутствует, так как она уже осуществлена.

## 2. Разбейте данные на выборки

In [347]:
from sklearn.model_selection import train_test_split

In [348]:
# Splitting data to independent and target features
features = data.drop(columns=['is_ultra'])
target = data['is_ultra']

print(features.head())
print(target.head())

   calls  minutes  messages   mb_used
0     40   311.90        83  19915.42
1     85   516.75        56  22696.96
2     77   467.66        86  21060.45
3    106   745.53        81   8437.39
4     66   418.74         1  14502.75
0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64


In [349]:
# Splitting independent features to training, validation and test samples
train_features, validation_features = train_test_split(features, test_size=0.4, random_state=42)
validation_features, test_features = train_test_split(validation_features, test_size=0.5, random_state=42)

print('Train sample size - ', train_features.shape)
print('Validation sample size - ', validation_features.shape)
print('Test sample size - ', test_features.shape)

Train sample size -  (1928, 4)
Validation sample size -  (643, 4)
Test sample size -  (643, 4)


In [350]:
# Splitting target feature to training, validation and test samples
train_target, validation_target = train_test_split(target, test_size=0.4, random_state=42)
validation_target, test_target = train_test_split(validation_target, test_size=0.5, random_state=42)

print('Train target feature size - ', train_target.shape)
print('Validation target feature size - ', validation_target.shape)
print('Test target feature size - ', test_target.shape)

Train target feature size -  (1928,)
Validation target feature size -  (643,)
Test target feature size -  (643,)


### Выводы (шаг 2)

Данные были разделены на независимые и целевой (__is_ultra__) признаки. После чего данные были разделены на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

## 3. Исследуйте модели

### Логистическая регрессия

In [351]:
import time
from sklearn.linear_model import LogisticRegression

In [352]:
start_time = time.time()

# Model hyperparameters setup
model_logreg = LogisticRegression(random_state=42, max_iter=200)

# Model training
model_logreg.fit(train_features, train_target)

# Calculation of model accuracy with the validation sample
model_logreg.score(validation_features, validation_target)

print("{:.5} s".format(time.time() - start_time))

0.040999 s


### Дерево решений

In [353]:
from sklearn.tree import DecisionTreeClassifier

In [359]:
start_time = time.time()

max_accuracy = -1

# Model hyperparameters variation
for i in range(21):
    for j in range(2, 203, 10):
        for k in range(1, 41):
            # Model hyperparameters setup
            if i == 0:
                # Replacement max_depth = 0 with None
                model_decisiontree = DecisionTreeClassifier(random_state=42, max_depth=None, min_samples_split=j, min_samples_leaf=k)
            else:
                model_decisiontree = DecisionTreeClassifier(random_state=42, max_depth=i, min_samples_split=j, min_samples_leaf=k)
            
            # Model training
            model_decisiontree.fit(train_features, train_target)
            
            # Calculation of model accuracy with the validation sample
            accuracy = model_decisiontree.score(validation_features, validation_target)
            
            # Finding best (by accuracy) model 
            if accuracy > max_accuracy:
                max_accuracy = accuracy
                if i == 0:
                    max_i = None
                else:
                    max_i = i
                max_j = j
                max_k = k

                print(
                    'Maximum accuracy = {} with max_depth = {}, min_samples_split = {}, min_samples_leaf = {}, time - {:.5} s'
                    .format(max_accuracy, max_i, max_j, max_k, time.time() - start_time)
                )
print('Whole best model finding time - {:.5} s'.format(time.time() - start_time))

Maximum accuracy = 0.7278382581648523 with max_depth = None, min_samples_split = 2, min_samples_leaf = 1, time - 0.011999 s
Maximum accuracy = 0.7589424572317263 with max_depth = None, min_samples_split = 2, min_samples_leaf = 2, time - 0.022998 s
Maximum accuracy = 0.7620528771384136 with max_depth = None, min_samples_split = 2, min_samples_leaf = 3, time - 0.033998 s
Maximum accuracy = 0.7636080870917574 with max_depth = None, min_samples_split = 2, min_samples_leaf = 8, time - 0.089997 s
Maximum accuracy = 0.7776049766718507 with max_depth = None, min_samples_split = 2, min_samples_leaf = 9, time - 0.099997 s
Maximum accuracy = 0.7807153965785381 with max_depth = None, min_samples_split = 2, min_samples_leaf = 10, time - 0.11 s
Maximum accuracy = 0.7900466562986003 with max_depth = None, min_samples_split = 2, min_samples_leaf = 13, time - 0.137 s
Maximum accuracy = 0.7947122861586314 with max_depth = None, min_samples_split = 2, min_samples_leaf = 14, time - 0.146 s
Maximum accurac

### Случайный лес

In [360]:
from sklearn.ensemble import RandomForestClassifier

In [361]:
start_time = time.time()

max_accuracy = -1

# Model hyperparameters variation
for i in range(50, 101, 10):
    for j in range(11):
        for k in range(2, 103, 10):
            for l in range(1, 21):
                # Model hyperparameters setup
                if j == 0:
                     # Replacement max_depth = 0 with None
                    model_randomforest = RandomForestClassifier(random_state=42, n_estimators=i, max_depth=None, min_samples_split=k, min_samples_leaf=l, n_jobs=-1)
                else:
                    model_randomforest = RandomForestClassifier(random_state=42, n_estimators=i, max_depth=j, min_samples_split=k, min_samples_leaf=l, n_jobs=-1)
                
                # Model training
                model_randomforest.fit(train_features, train_target)
                
                # Calculation of model accuracy with the validation sample
                accuracy = model_randomforest.score(validation_features, validation_target)
                
                # Finding best (by accuracy) model 
                if accuracy > max_accuracy:
                    max_accuracy = accuracy
                    max_i = i
                    if j == 0:
                        max_j = None
                    else:
                        max_j = j
                    max_k = k
                    max_l = l

                    print(
                        'Maximum accuracy = {} with n_estimators = {}, max_depth = {}, min_samples_split = {}, min_samples_leaf = {}, time - {:.5} s'
                        .format(max_accuracy, max_i, max_j, max_k, max_l, time.time() - start_time)
                    )

print('Whole best model finding time -', time.time() - start_time)

Maximum accuracy = 0.7962674961119751 with n_estimators = 50, max_depth = None, min_samples_split = 2, min_samples_leaf = 1, time - 0.129 s
Maximum accuracy = 0.8040435458786936 with n_estimators = 50, max_depth = None, min_samples_split = 2, min_samples_leaf = 2, time - 0.24699 s
Maximum accuracy = 0.8055987558320373 with n_estimators = 50, max_depth = None, min_samples_split = 2, min_samples_leaf = 3, time - 0.36299 s
Maximum accuracy = 0.8118195956454122 with n_estimators = 50, max_depth = None, min_samples_split = 2, min_samples_leaf = 4, time - 0.48799 s
Maximum accuracy = 0.8164852255054432 with n_estimators = 50, max_depth = None, min_samples_split = 2, min_samples_leaf = 6, time - 0.71098 s
Maximum accuracy = 0.8211508553654744 with n_estimators = 50, max_depth = None, min_samples_split = 2, min_samples_leaf = 7, time - 0.81898 s
Maximum accuracy = 0.8227060653188181 with n_estimators = 80, max_depth = None, min_samples_split = 22, min_samples_leaf = 1, time - 904.83 s
Maximum 

### Выводы (шаг 3)

В ходе исследования были рассмотрены 3 модели:
- __Логистическая регрессия__. Модель обучается быстрее всего (41 мс) за счет отсутствия гиперпараметров, которые можно варьировать. Но и результат проверки модели на валидационной выборке худший - __accuracy__ около __0.74__;
- __Дерево решений__. Модель обучается значительно медленнее (3 с для лучшего результата и более 2 мин перебора вариантов гиперпараметров), чем логистическая регрессия, за счет варьирования гиперпараметров. Но и результат проверки модели на валидационной выборке значительно лучше - __accuracy__ около __0.81__. Лучшая модель получается при:
    - отсутствие ограничения на максимальную глубину дерева;
    - достаточно большом (около 100) значении минимального количества объектов, необходимых для разбиения;
    - достаточно небольшом (около 5) значении минимального количества объектов, необходимых для создания листа.
- __Случайны лес__. Модель достигает уровня 0.81 быстрее (0.5 с), чем дерево решений, за счет добавления количества деревьев в гиперпараметры, но общее время обучения у нее намного дольше (22 мин для лучшего результата, 0.8 с для результата очень близкого к лучшему и более 36 мин перебора вариантов гиперпараметров). Результат проверки модели на валидационной выборке чуть лучше, чем у дерева решений - __accuracy__ около __0.82__. Лучшая модель получается при:
    - количестве деревьев около 90;
    - отсутствие ограничения на максимальную глубину дерева;
    - достаточно небольшом (около 20) значении минимального количества объектов, необходимых для разбиения;
    - минимальном (1) значении минимального количества объектов, необходимых для создания листа.

Важно заметить, что варьирование параметров дерева решений дает больший эффект за меньшее время, чем у случайного леса. У дерева решений прирост accuracy составил около 0.9 (с 0.72 до 0.81), а у случайного леса accuracy сразу был значительно выше, но прирост составил около 0.3 (с 0.79 до 0.82).

Из этого можно сделать вывод, что __моделью с лучшим показателем accuracy__ на валидационной выборке является __случайный лес__. Он сразу имеет хорошие (по сравнению с другими моделями) показатели, а при долгом обучении может их увеличить.

Важно отметить, что для полноценного сравнения моделей, данного тестирования недостаточно, ввиду наличие лишь одной выборки и отсутствия времени на тестирование различных вариантов варьирования гиперпараметров.

## 4. Проверьте модель на тестовой выборке

In [362]:
# Best model hyperparameters setup
best_model = RandomForestClassifier(random_state=42, n_estimators=90, min_samples_split=22, n_jobs=-1)

In [363]:
# Best model training
best_model.fit(train_features.append(validation_features), train_target.append(validation_target))

RandomForestClassifier(min_samples_split=22, n_estimators=90, n_jobs=-1,
                       random_state=42)

In [364]:
# Calculation of model accuracy with the test sample
best_model.score(test_features, test_target)

0.8164852255054432

In [365]:
# Overfitting test
best_model.score(train_features.append(validation_features), train_target.append(validation_target))

0.8630882924931933

### Выводы (шаг 4)

Лучшей моделью стал случайный лес со следующими параметрами:
- n_estimators = 90;
- max_depth = None (стандартное значение);
- min_samples_split = 22;
- min_samples_leaf = 1 (стандартное значение).

Данная модель была обучена на выборке, объединенной из обучающей и валидационной выборок для улучшения ее показателей за счет максимизации обучающей выборки. После этого была рассчитана точность модели для тестовой выборки, которая составила __0.82__, что является достаточным результатом для выполнения данной работы, но я бы не назвал данный результат достаточным для реального применения данной модели. Также у этой модели наблюдается небольшая степень переобучения - accuracy для обучающей выборки составляет 0.86.

## 5. (бонус) Проверьте модели на адекватность

In [367]:
# Classes ratio
test_target.value_counts()

0    448
1    195
Name: is_ultra, dtype: int64

In [368]:
# Proportion of larger class
len(test_target[test_target == 0]) / len(test_target)

0.6967340590979783

### Выводы (шаг 5)

Исходя из того, что 70% тестовой выборки составляют пользователи тарифа Smart и 30% процентов пользователи тарифа Ultra. То если бы любая модель всегда выбирала тариф Smart, то была бы права в 70% случаев. Все из представленных моделей имеют показатель точности более 70%, что говорит об их адекватности.

## Чек-лист готовности проекта

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 4: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75